In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

c:\Users\shaur\anaconda3\envs\dennis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

# Change the current working directory
new_directory = "C:/Users/shaur/Downloads"
os.chdir(new_directory)

# Verify the change
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\shaur\Downloads


In [3]:
model_name = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_name).to(device)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
text = "The black cat sat on the couch and the brown dog slept on the rug."
tokenzed_text = tokenizer(text, return_tensors="pt")

In [6]:
inputs = {k:v.to(device) for k,v in tokenzed_text.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1878, -0.0283, -0.1687,  ..., -0.1363,  0.7483,  0.1205],
         [-0.2751,  0.0506, -0.4477,  ...,  0.1321,  1.3671, -0.4097],
         [ 0.0966,  0.1944, -0.0447,  ..., -0.4726,  0.7158, -0.3971],
         ...,
         [ 0.1167,  0.0275, -0.0336,  ..., -0.2981,  0.5688, -0.2564],
         [ 0.6191,  0.1539, -0.6147,  ...,  0.0474, -0.1249, -0.5419],
         [ 0.2106,  0.3435, -0.1342,  ..., -0.1020,  0.3287, -0.4919]]],
       device='cuda:0'), hidden_states=None, attentions=None)


In [7]:
black = outputs.last_hidden_state[:,2,:]
brown = outputs.last_hidden_state[:,10,:]

In [8]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))
cos_sim(black.cpu().squeeze(0).numpy(),brown.cpu().squeeze(0).numpy())

0.90681314

## Multi-head attention